In [1]:
import numpy as np

In [3]:
eta_vec = np.array([0.1, 0.25]) 
n_vec = np.array([1,2,3,4,5,6,7,8,9,10])
alpha_vec = np.array([-1.01, -1.25, -1.5, -1.75, -2., -5.])
num_prices = len(eta_vec) * len(n_vec) * len(alpha_vec)


In [26]:
def generic_CF(u, params, S0, r, q, T, model):
       
    if (model == 'BMS'):
       
        sigma = params[0]
        
        mu = np.log(S0) + (r - q - 0.5 * sigma**2) * T
        # characteristic function for BMS
        a = sigma*np.sqrt(T)
        phi = np.exp(1j*mu*u-(a*u)**2/2)
        return phi
    
    elif (model == 'Heston'):
        kappa = params[0]
        theta = params[1] 
        sigma = params[2]
        rho = params[3]
        v0 = params[4]
        
        tmp = (kappa - 1j * rho * sigma * u)
        g = np.sqrt(sigma**2 * (u**2 + 1j*u) + tmp**2)
        pow1 = 2 * kappa * theta / (sigma**2)
        numer1 = (kappa*theta*T*tmp)/(sigma**2) + 1j*u*T*r + 1j*u*np.log(S0)
        log_denum1 = pow1 * np.log(np.cosh(g*T/2)+(tmp/g)*np.sinh(g*T/2))
        tmp2 = ((u*u+1j*u)*v0)/(g/np.tanh(g*T/2)+tmp)
        log_phi = numer1 - log_denum1 - tmp2
        phi = np.exp(log_phi)
        return phi 
        
    elif (model == 'VG'):
        sigma = params[0]
        nu = params[1] 
        theta = params[2] 
        if (nu == 0):
            mu = np.log(S0) + (r-q - theta -0.5*sigma**2)*T
            phi  = np.exp(1j*u*mu) * np.exp((1j*theta*u-0.5*sigma**2*u**2)*T)
        else:
            mu  = np.log(S0) + (r-q + np.log(1-theta*nu-0.5*sigma**2*nu)/nu)*T
            phi = np.exp(1j*u*mu) * ((1 - 1j*nu*theta*u + 0.5*nu*sigma**2*u**2)**(-T/nu))
            
        return phi


In [28]:
def genericFFT(params, S0, K, r, q, T, alpha, eta, n, model):
    
    N = 2**n
    
    # step-size in log strike space
    lda = (2 * np.pi / N) / eta
    
    # choice of beta
    #beta = np.log(S0)-N*lda/2 # the log strike we want is in the middle of the array
    beta = np.log(K) # the log strike we want is the first element of the array
    
    # forming vector x and strikes km for m=1,...,N
    km = np.zeros(N)
    xX = np.zeros(N)
    
    # discount factor
    df = np.exp(-r*T)
    
    nuJ = np.arange(N) * eta
    psi_nuJ = generic_CF(nuJ - (alpha + 1) * 1j, params, S0, r, q, T, model) / ((alpha + 1j*nuJ)*(alpha+1+1j*nuJ))
    
    km = beta + lda * np.arange(N)
    w = eta * np.ones(N)
    w[0] = eta / 2
    xX = np.exp(-1j * beta * nuJ) * df * psi_nuJ * w
     
    yY = np.fft.fft(xX)
    cT_km = np.zeros(N) 
    multiplier = np.exp(-alpha * km) / np.pi
    cT_km = multiplier * np.real(yY)
    
    return km, cT_km

In [30]:
def price_all_puts(params, S0, K, r, q, T, model, alpha_vec, eta_vec, n_vec):
    num_prices = len(eta_vec) * len(n_vec) * len(alpha_vec)
    put_matrix = np.zeros([num_prices, 4])
    i = 0
    for eta in eta_vec:
        for n in n_vec:
            for alpha in alpha_vec:
                # pricing via fft
                put = 0
                k_vec, option_vec = genericFFT(params, S0, K, r, q, T, alpha, eta, n, model)
                put = option_vec[0]
                put_matrix[i] = np.array([eta, n, alpha, put])
                i += 1
    return put_matrix


In [32]:
S0 = 100
K = 80
r = 0.05
q = 0.01
T = 365 / 365
alpha = 1.5
eta = 0.25
n = 12

In [36]:
model = 'BMS'
params = [0.30]
bms_matrix = price_all_puts(params, S0, K, r, q, T, model, alpha_vec, eta_vec, n_vec)

print('Model = ' + model)
print('eta\tN\talpha\tput')
for i in range(num_prices):
    print('%.2f\t2^%i\t%.2f\t%.4f' % (bms_matrix[i,0], bms_matrix[i,1], bms_matrix[i,2], bms_matrix[i,3]))

Model = BMS
eta	N	alpha	put
0.10	2^1	-1.01	119.1631
0.10	2^1	-1.25	9.7204
0.10	2^1	-1.50	4.2160
0.10	2^1	-1.75	2.3602
0.10	2^1	-2.00	1.5078
0.10	2^1	-5.00	0.1558
0.10	2^2	-1.01	114.5662
0.10	2^2	-1.25	15.1418
0.10	2^2	-1.50	8.2398
0.10	2^2	-1.75	4.9887
0.10	2^2	-2.00	3.3015
0.10	2^2	-5.00	0.3618
0.10	2^3	-1.01	106.1077
0.10	2^3	-1.25	14.6654
0.10	2^3	-1.50	10.5837
0.10	2^3	-1.75	7.6018
0.10	2^3	-2.00	5.5697
0.10	2^3	-5.00	0.7595
0.10	2^4	-1.01	96.6429
0.10	2^4	-1.25	8.9770
0.10	2^4	-1.50	8.0310
0.10	2^4	-1.75	6.9977
0.10	2^4	-2.00	6.0092
0.10	2^4	-5.00	1.4482
0.10	2^5	-1.01	91.0149
0.10	2^5	-1.25	4.0813
0.10	2^5	-1.50	4.0987
0.10	2^5	-1.75	4.0506
0.10	2^5	-2.00	3.9516
0.10	2^5	-5.00	2.2744
0.10	2^6	-1.01	89.7431
0.10	2^6	-1.25	2.7396
0.10	2^6	-1.50	2.7569
0.10	2^6	-1.75	2.7701
0.10	2^6	-2.00	2.7789
0.10	2^6	-5.00	2.6727
0.10	2^7	-1.01	89.7316
0.10	2^7	-1.25	2.7079
0.10	2^7	-1.50	2.7080
0.10	2^7	-1.75	2.7080
0.10	2^7	-2.00	2.7080
0.10	2^7	-5.00	2.7079
0.10	2^8	-1.01	89.7316
0.10	2^8	-1.

In [40]:
model = 'Heston'
kappa = 2
theta = 0.05
sigma = 0.3
rho = -0.7
v0 = 0.04
params = [kappa, theta, sigma, rho, v0]
heston_matrix = price_all_puts(params, S0, K, r, q, T, model, alpha_vec, eta_vec, n_vec)

print('Model = ' + model)
print('eta\tN\talpha\tput')
for i in range(num_prices):
    print('%.2f\t2^%i\t%.2f\t%.4f' % (heston_matrix[i,0], heston_matrix[i,1], heston_matrix[i,2], heston_matrix[i,3]))

Model = Heston
eta	N	alpha	put
0.10	2^1	-1.01	119.0472
0.10	2^1	-1.25	9.6182
0.10	2^1	-1.50	4.1245
0.10	2^1	-1.75	2.2770
0.10	2^1	-2.00	1.4315
0.10	2^1	-5.00	0.1141
0.10	2^2	-1.01	114.2997
0.10	2^2	-1.25	14.9055
0.10	2^2	-1.50	8.0276
0.10	2^2	-1.75	4.7956
0.10	2^2	-2.00	3.1241
0.10	2^2	-5.00	0.2644
0.10	2^3	-1.01	105.5645
0.10	2^3	-1.25	14.1767
0.10	2^3	-1.50	10.1408
0.10	2^3	-1.75	7.1965
0.10	2^3	-2.00	5.1958
0.10	2^3	-5.00	0.5512
0.10	2^4	-1.01	95.6560
0.10	2^4	-1.25	8.0665
0.10	2^4	-1.50	7.1901
0.10	2^4	-1.75	6.2176
0.10	2^4	-2.00	5.2819
0.10	2^4	-5.00	1.0214
0.10	2^5	-1.01	89.4789
0.10	2^5	-1.25	2.6310
0.10	2^5	-1.50	2.7295
0.10	2^5	-1.75	2.7547
0.10	2^5	-2.00	2.7220
0.10	2^5	-5.00	1.4521
0.10	2^6	-1.01	88.0744
0.10	2^6	-1.25	1.1102
0.10	2^6	-1.50	1.1666
0.10	2^6	-1.75	1.2167
0.10	2^6	-2.00	1.2605
0.10	2^6	-5.00	1.3979
0.10	2^7	-1.01	88.3778
0.10	2^7	-1.25	1.3512
0.10	2^7	-1.50	1.3486
0.10	2^7	-1.75	1.3464
0.10	2^7	-2.00	1.3445
0.10	2^7	-5.00	1.3390
0.10	2^8	-1.01	88.3649
0.10	2^8	

In [44]:
model = 'VG'

vg_matrix = price_all_puts(params, S0, K, r, q, T, model, alpha_vec, eta_vec, n_vec)

print('Model = ' + model)
print('eta\tN\talpha\tput')
for i in range(num_prices):
    print('%.2f\t2^%i\t%.2f\t%.4f' % (vg_matrix[i,0], vg_matrix[i,1], vg_matrix[i,2], vg_matrix[i,3]))

Model = VG
eta	N	alpha	put
0.10	2^1	-1.01	126.7667
0.10	2^1	-1.25	19.5874
0.10	2^1	-1.50	19.8332
0.10	2^1	-1.75	33.7867
0.10	2^1	-2.00	86.2819
0.10	2^1	-5.00	-44596639.4192
0.10	2^2	-1.01	131.4415
0.10	2^2	-1.25	36.3340
0.10	2^2	-1.50	39.6408
0.10	2^2	-1.75	61.0876
0.10	2^2	-2.00	126.4262
0.10	2^2	-5.00	7075816.7341
0.10	2^3	-1.01	136.2420
0.10	2^3	-1.25	48.8483
0.10	2^3	-1.50	51.9951
0.10	2^3	-1.75	58.7252
0.10	2^3	-2.00	60.3157
0.10	2^3	-5.00	3103.0577
0.10	2^4	-1.01	137.7129
0.10	2^4	-1.25	50.7198
0.10	2^4	-1.50	50.7386
0.10	2^4	-1.75	50.6420
0.10	2^4	-2.00	50.5640
0.10	2^4	-5.00	7726.4848
0.10	2^5	-1.01	137.7254
0.10	2^5	-1.25	50.7017
0.10	2^5	-1.50	50.7017
0.10	2^5	-1.75	50.7017
0.10	2^5	-2.00	50.7017
0.10	2^5	-5.00	7730.8680
0.10	2^6	-1.01	137.7254
0.10	2^6	-1.25	50.7017
0.10	2^6	-1.50	50.7017
0.10	2^6	-1.75	50.7017
0.10	2^6	-2.00	50.7017
0.10	2^6	-5.00	7730.8680
0.10	2^7	-1.01	137.7254
0.10	2^7	-1.25	50.7017
0.10	2^7	-1.50	50.7017
0.10	2^7	-1.75	50.7017
0.10	2^7	-2.00	50.7017
0.